# Predikcia kvality vína
V tomto cvičení preskúmame, ako použiť strojové učenie na predikciu kvality vína na základe jeho chemických vlastností. Prejdeme si základné kroky pracovného postupu strojového učenia vrátane načítania dát, predspracovania, trénovania modelu a hodnotenia.

## Časť 1: Načítanie dát
Najprv musíme načítať náš dataset. Použijeme dataset kvality vína, ktorý obsahuje chemické vlastnosti vína a hodnotenie kvality.

<details>
  <summary>Hint (click to expand)</summary>
  Pri načítaní dát použite knižnicu Pandas. Funkcia `read_csv` je užitočná na načítanie dát z CSV súborov.
</details>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression

# Načítanie datasetu
wine_data = pd.read_csv('data/winequality-red.csv', sep=';')

# Zobrazíme prvých 5 riadkov datasetu
print(wine_data.head())

Úloha: Zobrazte prvých 5 riadkov datasetu, aby ste pochopili jeho štruktúru. Použite `wine_data.head()`.

Reflexná otázka: Prečo je dôležité preskúmať dáta pred ich ďalším spracovaním? Ako môže formát dát ovplyvniť ich načítanie?

## Časť 2: Prieskum dát
Predtým, než prejdeme na predspracovanie dát, preskúmajme trochu náš dataset.

<details>
  <summary>Hint (click to expand)</summary>
  Vizualizácia distribúcie cieľovej premennej môže poskytnúť užitočné informácie o jej rozložení. Použite knižnicu Seaborn na vykreslenie grafu.
</details>

In [2]:
# Zobrazenie základných informácií o datasete
wine_data.info()
plt.figure(figsize=(10, 6))
sns.countplot(x='quality', data=wine_data)
plt.title('Distribution of Wine Quality')
plt.show()

Úloha: Vykreslite distribúciu premennej 'quality', aby ste videli, ako je distribuovaná kvalita vína. Použite knižnice ako `matplotlib` alebo `seaborn` pre vizualizáciu.

Reflexná otázka: Aké zistenia ste urobili z vizualizácie distribúcie kvality vína? Ako by tieto informácie mohli ovplyvniť výber a trénovanie modelu strojového učenia?

## Časť 3: Predspracovanie dát
### Riešenie chýbajúcich hodnôt
Najprv skontrolujeme chýbajúce hodnoty a zaoberáme sa nimi, ak nejaké sú.

In [3]:
# Kontrola chýbajúcich hodnôt
print(wine_data.isnull().sum())

# V tomto riešení nemáme žiadne chýbajúce hodnoty

### Škálovanie prvkov
Mnoho modelov strojového učenia lepšie funguje, keď sú vstupné numerické premenné škálované alebo normalizované.

<details>
  <summary>Hint (click to expand)</summary>
  Pred trénovaním modelu je často potrebné dáta normalizovať alebo škálovať. Použite `StandardScaler` z knižnice Scikit-learn na škálovanie funkcií.
</details>

In [4]:
from sklearn.preprocessing import StandardScaler

# Škálovanie
X = wine_data.drop('quality', axis=1)
y = wine_data['quality']

# Škálovanie prvkov
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

Úloha: Škálujte prvky a overte to zobrazením prvých 5 riadkov škálovaných prvkov.

Reflexná otázka: Prečo je škálovanie prvkov dôležité pri trénovaní modelov strojového učenia? Ako by mohlo ovplyvniť výsledky, keby sme dáta neškálovali?

## Časť 4: Rozdelenie datasetu
Rozdelíme dataset na trénovaciu a testovaciu množinu.

<details>
  <summary>Hint (click to expand)</summary>
  Rozdelenie dát na trénovaciu a testovaciu sadu pomáha overiť, ako dobre model funguje na nevidených dátach.
</details>

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

Reflexná otázka: Ako pomáha rozdelenie dát na trénovaciu a testovaciu sadu zabrániť problému pretrénovania? Aký vplyv by malo  použitie príliš veľkej časti dát na trénovanie?

## Časť 5: Trénovanie modelu
Použijeme model Random Forest na predikciu kvality vína.

<details>
  <summary>Hint (click to expand)</summary>
  Random Forest je populárny model pre klasifikačné úlohy. Skúste ho použiť a sledujte, ako sa model učí z trénovacích dát.
</details>

In [6]:
model_rf = RandomForestClassifier(random_state=42)
model_rf.fit(X_train, y_train)

print("Training Accuracy (Random Forest):", model_rf.score(X_train, y_train))

Úloha: Natrénujte model a vypíšte skóre trénovania.

Reflexná otázka: Prečo sme si vybrali model Random Forest pre túto úlohu? Aké sú jeho výhody oproti iným modelom, ktoré poznáte?

## Časť 6: Hodnotenie modelu
Nakoniec, poďme zhodnotiť náš model na testovacej množine.

<details>
  <summary>Hint (click to expand)</summary>
  Na hodnotenie modelu použite metriky ako presnosť a správu o klasifikácii. Tieto metriky poskytujú prehľad o výkone modelu.
</details>

In [7]:
y_pred_rf = model_rf.predict(X_test)
print("Accuracy (Random Forest):", accuracy_score(y_test, y_pred_rf))
print("\nClassification Report (Random Forest):\n", classification_report(y_test, y_pred_rf))

Úloha: Ohodnoťte výkonnosť modelu a zvážte experimentovanie s rôznymi modelmi alebo technikami predspracovania, aby ste videli, či môžete zlepšiť presnosť.

Reflexná otázka: Ako hodnotíte výkon modelu Random Forest na základe výsledkov? Sú miesta, kde by model mohol byť zlepšený?

## Časť 7: Experimentovanie a záver
Vyskúšajte si rôzne modely strojového učenia, techniky predspracovania alebo metódy inžinierstva prvkov na zlepšenie výkonnosti modelu.

Úloha: Vyberte aspoň jeden nový model na trénovanie a porovnajte jeho výkonnosť s modelom Random Forest. Zdokumentujte svoje zistenia a závery.

<details>
  <summary>Hint (click to expand)</summary>
  Skúste rôzne strojové učenie modely, techniky predspracovania alebo metódy inžinierstva prvkov na zlepšenie výkonnosti modelu.
</details>

In [8]:
# Logistic Regression
model_lr = LogisticRegression(max_iter=10000, random_state=42)
model_lr.fit(X_train, y_train)

# Trénovacie skóre
print("Training Accuracy (Logistic Regression):", model_lr.score(X_train, y_train))

# Model Evaluation (Logistic Regression)
y_pred_lr = model_lr.predict(X_test)
print("Accuracy (Logistic Regression):", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report (Logistic Regression):\n", classification_report(y_test, y_pred_lr))